In [104]:
import os
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
import pytz
import random
from pysolar import solar
import datetime
import reverse_geocoder as rg 
from wwo_hist import retrieve_hist_data

In [147]:
tz = pytz.timezone("Turkey")
now = datetime.datetime.now(tz).replace(microsecond=0,second=0,minute=0)
date_format = "%b %d %Y %H:00:00"
total_minutes_in_year = 60*24*365
total_solar_panel_area = 1
minute_diff = 60


#Turkey's latitude and longitude ranges
min_latitude = 35.9025
max_latitude = 42.0268
min_longitude = 25.9090
max_longitude = 44.5742

In [148]:
def get_altitude_by_location(lat,lon,date):
    return solar.get_altitude(lat, lon, date)

def get_azimuth_by_location(lat,lon,date):
    return solar.get_azimuth(lat, lon, date)

def get_radiation_by_location(lat,lon,date):
    return solar.radiation.get_radiation_direct(date, get_altitude_by_location(lat,lon,date))

In [149]:
def get_solar_info(lat,lon,date):
    return get_altitude_by_location(lat,lon,date),\
            get_azimuth_by_location(lat,lon,date),\
            get_radiation_by_location(lat,lon,date)




In [150]:
def get_location(coordinates):
    return rg.search(coordinates) 

In [151]:
def get_random_location(min_lat,max_lat,min_lon,max_lon):
     return random.uniform(min_lat,max_lat), random.uniform(min_lon,max_lon)

location = get_random_location(min_latitude,max_latitude,min_longitude,max_longitude)
print("Coordinates", location)
result = get_location(location)
print("City name is" , result[0]["admin1"], "\nCounty name is" ,result[0]["name"])
latitude, longitude = location[0],location[1]

Coordinates (40.17075198379227, 37.83957113026611)
City name is Sivas 
County name is Serefiye


In [153]:
dates = []
solar_info = []
for min_diff in range(total_minutes_in_year,0,minute_diff*-1):
    past_date = now - datetime.timedelta(minutes=min_diff)
    dates.append(past_date)
    solar_info.append(get_solar_info(latitude,longitude,past_date))

In [154]:
df = pd.concat([pd.DataFrame(dates,columns=["date"]),\
                pd.DataFrame(solar_info,columns=["altitude","azimuth","solar_radiation"])],\
               axis=1)

In [155]:
df.set_index("date",inplace=True)

In [156]:
frequency = 1
start_date = now - datetime.timedelta(minutes=total_minutes_in_year)
end_date = now
api_key = os.environ['WWO_API_KEY'] 
location_list = [str(latitude) +  "," + str(longitude)]

In [169]:
hist_data = retrieve_hist_data(api_key,
                                location_list,
                                start_date,
                                end_date,
                                frequency,
                                location_label = False,
                                store_df = True)



Retrieving weather data for 40.17075198379227,37.83957113026611


Currently retrieving data for 40.17075198379227,37.83957113026611: from 2018-12-22 to 2018-12-31
Time elapsed (hh:mm:ss.ms) 0:00:00.701595
Currently retrieving data for 40.17075198379227,37.83957113026611: from 2019-01-01 to 2019-01-31
Time elapsed (hh:mm:ss.ms) 0:00:01.867295
Currently retrieving data for 40.17075198379227,37.83957113026611: from 2019-02-01 to 2019-02-28
Time elapsed (hh:mm:ss.ms) 0:00:02.859866
Currently retrieving data for 40.17075198379227,37.83957113026611: from 2019-03-01 to 2019-03-31
Time elapsed (hh:mm:ss.ms) 0:00:04.071018
Currently retrieving data for 40.17075198379227,37.83957113026611: from 2019-04-01 to 2019-04-30
Time elapsed (hh:mm:ss.ms) 0:00:05.817085
Currently retrieving data for 40.17075198379227,37.83957113026611: from 2019-05-01 to 2019-05-31
Time elapsed (hh:mm:ss.ms) 0:00:07.331959
Currently retrieving data for 40.17075198379227,37.83957113026611: from 2019-06-01 to 2019-06-30
T

In [170]:
historical = pd.DataFrame(hist_data[0])

In [172]:
historical.set_index("date_time",inplace=True)

In [173]:
historical.index = historical.index.tz_localize(tz)
historical

,maxtempC,mintempC,totalSnow_cm,sunHour,uvIndex,uvIndex,moon_illumination,moonrise,moonset,sunrise,sunset,DewPointC,FeelsLikeC,HeatIndexC,WindChillC,WindGustKmph,cloudcover,humidity,precipMM,pressure,tempC,visibility,winddirDegree,windspeedKmph
date_time,,,,,,,,,,,,,,,,,,,,,,,,
2018-12-22 00:00:00+03:00,2,-1,0.0,8.7,2,0,100,05:03 PM,07:05 AM,07:48 AM,05:07 PM,-3,-3,-1,-3,10,46,88,0.0,1025,-1,10,205,5
2018-12-22 01:00:00+03:00,2,-1,0.0,8.7,2,0,100,05:03 PM,07:05 AM,07:48 AM,05:07 PM,-3,-3,-1,-3,9,45,87,0.0,1025,-1,10,207,5
2018-12-22 02:00:00+03:00,2,-1,0.0,8.7,2,0,100,05:03 PM,07:05 AM,07:48 AM,05:07 PM,-3,-2,-1,-2,8,44,87,0.0,1025,-1,10,208,4
2018-12-22 03:00:00+03:00,2,-1,0.0,8.7,2,0,100,05:03 PM,07:05 AM,07:48 AM,05:07 PM,-3,-2,-1,-2,7,43,87,0.0,1025,-1,10,210,4
2018-12-22 04:00:00+03:00,2,-1,0.0,8.7,2,0,100,05:03 PM,07:05 AM,07:48 AM,05:07 PM,-3,-2,-1,-2,8,62,86,0.0,1025,-1,10,218,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-12-22 19:00:00+03:00,9,1,0.0,8.7,1,1,21,03:24 AM,02:33 PM,07:48 AM,05:06 PM,-3,1,3,1,18,6,63,0.0,1017,3,10,120,9
2019-12-22 20:00:00+03:00,9,1,0.0,8.7,1,1,21,03:24 AM,02:33 PM,07:48 AM,05:06 PM,-3,0,3,0,19,6,65,0.0,1017,3,10,119,9
2019-12-22 21:00:00+03:00,9,1,0.0,8.7,1,1,21,03:24 AM,02:33 PM,07:48 AM,05:06 PM,-3,-1,2,-1,19,7,67,0.0,1018,2,10,117,9


In [174]:
df.index

DatetimeIndex(['2018-12-22 17:00:00+03:00', '2018-12-22 18:00:00+03:00', '2018-12-22 19:00:00+03:00', '2018-12-22 20:00:00+03:00', '2018-12-22 21:00:00+03:00', '2018-12-22 22:00:00+03:00', '2018-12-22 23:00:00+03:00', '2018-12-23 00:00:00+03:00', '2018-12-23 01:00:00+03:00', '2018-12-23 02:00:00+03:00',
               ...
               '2019-12-22 07:00:00+03:00', '2019-12-22 08:00:00+03:00', '2019-12-22 09:00:00+03:00', '2019-12-22 10:00:00+03:00', '2019-12-22 11:00:00+03:00', '2019-12-22 12:00:00+03:00', '2019-12-22 13:00:00+03:00', '2019-12-22 14:00:00+03:00', '2019-12-22 15:00:00+03:00', '2019-12-22 16:00:00+03:00'], dtype='datetime64[ns, Turkey]', name='date', length=8760, freq=None)

In [178]:
data = pd.merge(df,historical,left_index=True,right_index=True)

In [179]:
data[:30]

,altitude,azimuth,solar_radiation,maxtempC,mintempC,totalSnow_cm,sunHour,uvIndex,uvIndex,moon_illumination,moonrise,moonset,sunrise,sunset,DewPointC,FeelsLikeC,HeatIndexC,WindChillC,WindGustKmph,cloudcover,humidity,precipMM,pressure,tempC,visibility,winddirDegree,windspeedKmph
2018-12-22 17:00:00+03:00,0.660708,238.422965,0.000000,2,-1,0.0,8.7,2,2,100,05:03 PM,07:05 AM,07:48 AM,05:07 PM,-3,-2,-1,-2,10,32,85,0.0,1024,-1,10,218,5
2018-12-22 18:00:00+03:00,-9.994299,247.683146,0.000000,2,-1,0.0,8.7,2,0,100,05:03 PM,07:05 AM,07:48 AM,05:07 PM,-3,-3,-1,-3,10,25,86,0.0,1024,-1,10,209,5
2018-12-22 19:00:00+03:00,-20.884933,256.410933,0.000000,2,-1,0.0,8.7,2,0,100,05:03 PM,07:05 AM,07:48 AM,05:07 PM,-3,-3,-1,-3,11,31,86,0.0,1024,-1,10,200,6
2018-12-22 20:00:00+03:00,-32.183148,265.200080,0.000000,2,-1,0.0,8.7,2,0,100,05:03 PM,07:05 AM,07:48 AM,05:07 PM,-3,-3,-1,-3,12,38,86,0.0,1025,-1,10,192,6
2018-12-22 21:00:00+03:00,-43.626992,274.914248,0.000000,2,-1,0.0,8.7,2,0,100,05:03 PM,07:05 AM,07:48 AM,05:07 PM,-3,-4,-1,-4,13,44,86,0.0,1025,-1,10,183,7
2018-12-22 22:00:00+03:00,-54.862995,287.171733,0.000000,2,-1,0.0,8.7,2,0,100,05:03 PM,07:05 AM,07:48 AM,05:07 PM,-4,-4,-1,-4,12,35,85,0.0,1024,-1,10,184,6
2018-12-22 23:00:00+03:00,-65.145248,305.748783,0.000000,2,-1,0.0,8.7,2,0,100,05:03 PM,07:05 AM,07:48 AM,05:07 PM,-4,-4,-1,-4,12,25,84,0.0,1024,-1,10,185,6
2018-12-23 00:00:00+03:00,-72.298517,338.964163,0.000000,4,-1,0.6,8.7,2,0,100,06:03 PM,08:12 AM,07:49 AM,05:07 PM,-4,-4,-2,-4,11,16,83,0.0,1023,-1,10,186,6
2018-12-23 01:00:00+03:00,-71.902070,24.794934,0.000000,4,-1,0.6,8.7,2,0,100,06:03 PM,08:12 AM,07:49 AM,05:07 PM,-4,-4,-2,-4,12,13,81,0.0,1023,-1,10,179,6
2018-12-23 02:00:00+03:00,-64.308405,56.334952,0.000000,4,-1,0.6,8.7,2,0,100,06:03 PM,08:12 AM,07:49 AM,05:07 PM,-5,-4,-2,-4,12,11,80,0.0,1023,-1,10,173,6


In [180]:
data.columns

Index(['altitude', 'azimuth', 'solar_radiation', 'maxtempC', 'mintempC', 'totalSnow_cm', 'sunHour', 'uvIndex', 'uvIndex', 'moon_illumination', 'moonrise', 'moonset', 'sunrise', 'sunset', 'DewPointC', 'FeelsLikeC', 'HeatIndexC', 'WindChillC', 'WindGustKmph', 'cloudcover', 'humidity', 'precipMM', 'pressure', 'tempC', 'visibility', 'winddirDegree', 'windspeedKmph'], dtype='object')